In [2]:
import metview as mv
import datetime
from ecmwf.opendata import Client

## Cara debug isi file GRIB

Sebelum ambil datanya, coba lihat dulu isi GRIB dengan cfgrib:

In [3]:
import cfgrib

# Periksa semua message group di file
print(cfgrib.open_datasets("medium-2t-wind-2025-09-02.grib2"))

Ignoring index file 'medium-2t-wind-2025-09-02.grib2.5b7b6.idx' incompatible with GRIB file
Ignoring index file 'medium-2t-wind-2025-09-02.grib2.5b7b6.idx' incompatible with GRIB file
/home/prema/miniconda3/envs/notebooks/lib/python3.13/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version, xarray will not decode the variable 'step' into a timedelta64 dtype based on the presence of a timedelta-like 'units' attribute by default. Instead it will rely on the presence of a timedelta64 'dtype' attribute, which is now xarray's default way of encoding timedelta64 values.
To continue decoding into a timedelta64 dtype, either set `decode_timedelta=True` when opening this dataset, or add the attribute `dtype='timedelta64[ns]'` to this variable on disk.
To opt-in to future behavior, set `decode_timedelta=False`.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
Ignoring index file 'medium-2t-wind-2025-09-02.grib2.5b7b6.idx' incompatible with GRIB file
/home/

[<xarray.Dataset> Size: 33MB
Dimensions:            (step: 4, latitude: 721, longitude: 1440)
Coordinates:
    time               datetime64[ns] 8B 2025-09-02
  * step               (step) timedelta64[ns] 32B 00:00:00 06:00:00 ... 18:00:00
    heightAboveGround  float64 8B 10.0
  * latitude           (latitude) float64 6kB 90.0 89.75 89.5 ... -89.75 -90.0
  * longitude          (longitude) float64 12kB -180.0 -179.8 ... 179.5 179.8
    valid_time         (step) datetime64[ns] 32B 2025-09-02 ... 2025-09-02T18...
Data variables:
    u10                (step, latitude, longitude) float32 17MB ...
    v10                (step, latitude, longitude) float32 17MB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts, <xarray.Dataset> Size: 17M

# Proses data ke CSV

# 1. Gabungkan semua file jadi satu dataset
Sementara ini baru 3 hari data, tapi nanti kalau sudah 1 bulan bisa langsung gabung semua file jadi satu dataset.

In [8]:
import cfgrib
import xarray as xr
from glob import glob

files = sorted(glob("medium-2t-wind*.grib2"))

datasets = []
for f in files:
    groups = cfgrib.open_datasets(f)

    ds_parts = []
    for g in groups:
        vars_in_g = list(g.data_vars)
        if any(v in vars_in_g for v in ["u10", "v10", "t2m"]):
            if "heightAboveGround" in g.coords:
                g = g.drop_vars("heightAboveGround")
            ds_parts.append(g)

    if ds_parts:
        ds = xr.merge(ds_parts, compat="override")  # force merge
        datasets.append(ds)

if datasets:
    ds_all = xr.concat(datasets, dim="time")
    print(ds_all)
    ds_all.to_netcdf("ecmwf_monthly.nc")
else:
    raise RuntimeError("Tidak ada dataset yang berhasil dimuat!")

Ignoring index file 'medium-2t-wind-2025-09-02.grib2.5b7b6.idx' incompatible with GRIB file
Ignoring index file 'medium-2t-wind-2025-09-02.grib2.5b7b6.idx' incompatible with GRIB file
/home/prema/miniconda3/envs/notebooks/lib/python3.13/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version, xarray will not decode the variable 'step' into a timedelta64 dtype based on the presence of a timedelta-like 'units' attribute by default. Instead it will rely on the presence of a timedelta64 'dtype' attribute, which is now xarray's default way of encoding timedelta64 values.
To continue decoding into a timedelta64 dtype, either set `decode_timedelta=True` when opening this dataset, or add the attribute `dtype='timedelta64[ns]'` to this variable on disk.
To opt-in to future behavior, set `decode_timedelta=False`.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
Ignoring index file 'medium-2t-wind-2025-09-02.grib2.5b7b6.idx' incompatible with GRIB file
/home/

<xarray.Dataset> Size: 150MB
Dimensions:     (time: 3, step: 4, latitude: 721, longitude: 1440)
Coordinates:
  * step        (step) timedelta64[ns] 32B 00:00:00 06:00:00 12:00:00 18:00:00
  * latitude    (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 12kB -180.0 -179.8 -179.5 ... 179.5 179.8
  * time        (time) datetime64[ns] 24B 2025-09-02 2025-09-03 2025-09-04
    valid_time  (time, step) datetime64[ns] 96B 2025-09-02 ... 2025-09-04T18:...
Data variables:
    u10         (time, step, latitude, longitude) float32 50MB -4.03 ... -0.9367
    v10         (time, step, latitude, longitude) float32 50MB 6.054 ... 2.113
    t2m         (time, step, latitude, longitude) float32 50MB 268.6 ... 216.8
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:     

# 2. Ambil variabel

In [ ]:
import pandas as pd

# Lokasi Jepang
locations = {
    "Tokyo": {"lat": 35.7, "lon": 139.7},
    "Hokkaido": {"lat": 43.1, "lon": 141.3},
    "Tohoku": {"lat": 38.3, "lon": 141.0},
    # Tambah lokasi lain jika perlu
}

dfs = []

for name, coords in locations.items():
    point = ds_all.sel(latitude=coords["lat"], longitude=coords["lon"], method="nearest")
    df_point = point.to_dataframe().reset_index()
    df_point["location"] = name  # tambahkan nama lokasi
    dfs.append(df_point)

# Gabung semua lokasi jadi 1 tabel
df_japan = pd.concat(dfs)

# Simpan ke CSV
df_japan.to_csv("ecmwf_japan.csv", index=False)

print("CSV saved:", df_japan.shape)

CSV saved: (36, 9)


Tiap daerah ada 12 baris (4 step waktu x 3 hari).

In [10]:
import pandas as pd

# Baca CSV
df = pd.read_csv("ecmwf_japan.csv")
df

,time,step,u10,v10,t2m,latitude,longitude,valid_time,location
0,2025-09-02,0 days 00:00:00,0.250824,1.351303,305.65082,35.75,139.75,2025-09-02 00:00:00,Tokyo
1,2025-09-02,0 days 06:00:00,-1.014755,4.196457,308.50287,35.75,139.75,2025-09-02 06:00:00,Tokyo
2,2025-09-02,0 days 12:00:00,0.077988,2.339233,303.81546,35.75,139.75,2025-09-02 12:00:00,Tokyo
3,2025-09-02,0 days 18:00:00,0.361191,1.763275,301.63373,35.75,139.75,2025-09-02 18:00:00,Tokyo
4,2025-09-03,0 days 00:00:00,-0.580078,-0.614441,306.06964,35.75,139.75,2025-09-03 00:00:00,Tokyo
5,2025-09-03,0 days 06:00:00,-0.869568,1.766067,307.92468,35.75,139.75,2025-09-03 06:00:00,Tokyo
6,2025-09-03,0 days 12:00:00,-1.149887,0.215729,303.23900,35.75,139.75,2025-09-03 12:00:00,Tokyo
7,2025-09-03,0 days 18:00:00,0.628143,-2.607620,299.25244,35.75,139.75,2025-09-03 18:00:00,Tokyo
8,2025-09-04,0 days 00:00:00,-2.173767,-0.887283,301.82687,35.75,139.75,2025-09-04 00:00:00,Tokyo
9,2025-09-04,0 days 06:00:00,0.304642,-0.087463,301.33557,35.75,139.75,2025-09-04 06:00:00,Tokyo
